In [1]:
%matplotlib inline
import sys
import random
import operator
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn import preprocessing, metrics, ensemble, neighbors, linear_model, tree, model_selection
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import manifold, decomposition
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection

/home/dj/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.columns

Index(['transaction_id', 'num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_1', 'cat_var_2',
       'cat_var_3', 'cat_var_4', 'cat_var_5', 'cat_var_6', 'cat_var_7',
       'cat_var_8', 'cat_var_9', 'cat_var_10', 'cat_var_11', 'cat_var_12',
       'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16', 'cat_var_17',
       'cat_var_18', 'cat_var_19', 'cat_var_20', 'cat_var_21', 'cat_var_22',
       'cat_var_23', 'cat_var_24', 'cat_var_25', 'cat_var_26', 'cat_var_27',
       'cat_var_28', 'cat_var_29', 'cat_var_30', 'cat_var_31', 'cat_var_32',
       'cat_var_33', 'cat_var_34', 'cat_var_35', 'cat_var_36', 'cat_var_37',
       'cat_var_38', 'cat_var_39', 'cat_var_40', 'cat_var_41', 'cat_var_42',
       'target'],
      dtype='object')

In [52]:
train.nunique()

transaction_id    348978
num_var_1          13385
num_var_2           5550
num_var_3              8
num_var_4           1006
num_var_5           4622
num_var_6          11827
num_var_7          26213
cat_var_1            534
cat_var_2             60
cat_var_3            616
cat_var_4              2
cat_var_5              2
cat_var_6            518
cat_var_7             20
cat_var_8            462
cat_var_9              5
cat_var_10            23
cat_var_11             5
cat_var_12             5
cat_var_13            52
cat_var_14            12
cat_var_15             2
cat_var_16             2
cat_var_17             2
cat_var_18             2
cat_var_19             2
cat_var_20             2
cat_var_21             2
cat_var_22             2
cat_var_23             2
cat_var_24             2
cat_var_25             2
cat_var_26             2
cat_var_27             2
cat_var_28             2
cat_var_29             2
cat_var_30             2
cat_var_31             1
cat_var_32             2


In [4]:
train_y=train['target']

In [53]:
all_data = pd.concat((train.loc[:,'num_var_1':'cat_var_42'],
                      test.loc[:,'num_var_1':'cat_var_42']))

In [54]:
#g = sns.distplot(all_data["num_var_7"], color="m", label="Skewness : %.2f"%(all_data["num_var_7"].skew()))
#g.legend(loc="best")

In [55]:
all_data.cat_var_18.value_counts()

ep    497666
tn    374778
Name: cat_var_18, dtype: int64

In [56]:
col_to_drop=['cat_var_24', 'cat_var_25', 'cat_var_26', 'cat_var_27', 'cat_var_28', 'cat_var_29', 'cat_var_30', 
             'cat_var_31', 'cat_var_32', 'cat_var_33', 'cat_var_34', 'cat_var_35', 'cat_var_36', 'cat_var_37', 
             'cat_var_38', 'cat_var_39', 'cat_var_40', 'cat_var_41','cat_var_42', 'num_var_3']

In [57]:
all_data=all_data.drop(col_to_drop, 1)

In [58]:
all_data.columns

Index(['num_var_1', 'num_var_2', 'num_var_4', 'num_var_5', 'num_var_6',
       'num_var_7', 'cat_var_1', 'cat_var_2', 'cat_var_3', 'cat_var_4',
       'cat_var_5', 'cat_var_6', 'cat_var_7', 'cat_var_8', 'cat_var_9',
       'cat_var_10', 'cat_var_11', 'cat_var_12', 'cat_var_13', 'cat_var_14',
       'cat_var_15', 'cat_var_16', 'cat_var_17', 'cat_var_18', 'cat_var_19',
       'cat_var_20', 'cat_var_21', 'cat_var_22', 'cat_var_23'],
      dtype='object')

In [59]:
def na_checker(data):   
    '''Check all columns in a dataframe and returns a list of columns to discard'''
    #Initialize vectors
    col_to_discard = []
    col_to_fill = []
    # Set conditions
    for i in data.columns:
        p = round(1-(float(data[i].count())/float(len(data[i]))),3) 
        if p > 0.75:
            col_to_discard.append((p,i))
        elif p <= 0.75 and p > 0:
            col_to_fill.append((p,i))
    # Print report
    print("Discard following columns: ", col_to_discard)
    print("Consider filling those columns: ", col_to_fill)
    
    # Return
    col = list(map(lambda x: x[1],col_to_discard))
    return col

In [60]:
def no_of_category(data):
    no_of_category = []
    for i in data.columns:
        if(data.dtypes[i]=='O'):
            no_of_category.append((i,len(data[i].unique())))
            
    print(pd.DataFrame(no_of_category))  

In [61]:
all_data['cat_var_1']=all_data['cat_var_1'].fillna('Nan')
all_data['cat_var_3']=all_data['cat_var_3'].fillna('Nan')
all_data['cat_var_6']=all_data['cat_var_6'].fillna('Nan')
all_data['cat_var_8']=all_data['cat_var_8'].fillna('Nan')

In [62]:
encoder=all_data.groupby('cat_var_1').size()
encoder=encoder/len(all_data)
all_data['cat_var_1_freq']=all_data.cat_var_1.map(encoder)

In [63]:
encoder=all_data.groupby('cat_var_2').size()
encoder=encoder/len(all_data)
all_data['cat_var_2_freq']=all_data.cat_var_2.map(encoder)

In [64]:
encoder=all_data.groupby('cat_var_3').size()
encoder=encoder/len(all_data)
all_data['cat_var_3_freq']=all_data.cat_var_3.map(encoder)

In [65]:
encoder=all_data.groupby('cat_var_6').size()
encoder=encoder/len(all_data)
all_data['cat_var_6_freq']=all_data.cat_var_6.map(encoder)

In [66]:
encoder=all_data.groupby('cat_var_7').size()
encoder=encoder/len(all_data)
all_data['cat_var_7_freq']=all_data.cat_var_7.map(encoder)

In [67]:
encoder=all_data.groupby('cat_var_8').size()
encoder=encoder/len(all_data)
all_data['cat_var_8_freq']=all_data.cat_var_8.map(encoder)

In [68]:
encoder=all_data.groupby('cat_var_10').size()
encoder=encoder/len(all_data)
all_data['cat_var_10_freq']=all_data.cat_var_10.map(encoder)

In [69]:
encoder=all_data.groupby('cat_var_13').size()
encoder=encoder/len(all_data)
all_data['cat_var_13_freq']=all_data.cat_var_13.map(encoder)

In [70]:
encoder=all_data.groupby('cat_var_14').size()
encoder=encoder/len(all_data)
all_data['cat_var_14_freq']=all_data.cat_var_14.map(encoder)

In [71]:
all_data.head()

,num_var_1,num_var_2,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,cat_var_4,...,cat_var_23,cat_var_1_freq,cat_var_2_freq,cat_var_3_freq,cat_var_6_freq,cat_var_7_freq,cat_var_8_freq,cat_var_10_freq,cat_var_13_freq,cat_var_14_freq
0,2.302632e-08,0.040182,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,Nan,ce,db,ep,...,0,0.039514,0.667156,0.023058,0.016274,0.998761,0.135110,0.030682,0.025584,0.711649
1,7.965789e-06,0.157872,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,da,tn,zl,tn,...,0,0.290023,0.295931,0.009840,0.289718,0.998761,0.135110,0.036715,0.299744,0.711649
2,7.828947e-08,0.089140,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,gf,ce,gs,tn,...,0,0.379122,0.667156,0.004132,0.378800,0.998761,0.379122,0.092515,0.380945,0.711649
3,7.894737e-08,0.227239,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,Nan,ce,fy,ep,...,0,0.039514,0.667156,0.002698,0.005679,0.998761,0.019988,0.029980,0.016051,0.034106
4,3.321053e-06,0.160410,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,da,tn,zn,tn,...,0,0.290023,0.295931,0.000298,0.289718,0.998761,0.135110,0.049561,0.299744,0.711649


In [72]:
def encoder(data):
    '''Map the categorical variables to numbers to work with scikit learn'''
    for col in data.columns:
        if data.dtypes[col] == "O":
            le = preprocessing.LabelEncoder()
            data[col]=le.fit_transform(data[col])
            #data[col] = le.transform(data[col])
    return data

In [73]:
all_data=encoder(all_data)

In [74]:
train_X = all_data[:len(train)]
test_X = all_data[len(train):]

In [75]:
test_id=test['transaction_id']

In [79]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i,feat))
    outfile.close()

def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=10, eta=0.2):
    params = {}
    params["objective"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params["eta"] = eta
    params["subsample"] = 0.7
    params["min_child_weight"] = 1
    params["colsample_bytree"] = 0.7
    params["max_depth"] = dep

    params["silent"] = 1
    params["seed"] = seed_val
    #params["max_delta_step"] = 2
    #params["gamma"] = 0.5
    num_rounds = rounds

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    if feature_names is not None:
        create_feature_map(feature_names)
        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
        importance = model.get_fscore(fmap='xgb.fmap')
        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
        imp_df.to_csv("imp_feat.txt", index=False)

    pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    pred_test_y2 = model.predict(xgb.DMatrix(test_X2), ntree_limit=model.best_ntree_limit)

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        return pred_test_y, loss, pred_test_y2
    else:
        return pred_test_y, loss, pred_test_y2

def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=10, eta=0.2):
    params = {}
    params["objective"] = "binary"
    params['metric'] = 'auc'
    params["max_depth"] = dep
    params["min_data_in_leaf"] = 64
    params["learning_rate"] = eta
    params["bagging_fraction"] = 0.7
    params["feature_fraction"] = 0.7
    params["bagging_freq"] = 5
    params["bagging_seed"] = seed_val
    params["verbosity"] = 0
    #params["reg_alpha"] = 20
    #params["reg_lambda"] = 20
    num_rounds = rounds

    plst = list(params.items())
    lgtrain = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgtest = lgb.Dataset(test_X, label=test_y)
        model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtest], early_stopping_rounds=100, verbose_eval=20)
    else:
        lgtest = lgb.DMatrix(test_X)
        model = lgb.train(params, lgtrain, num_rounds)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        print(loss)
        return pred_test_y, loss, pred_test_y2
    else:
        return pred_test_y, loss, pred_test_y2


In [80]:
if __name__ == "__main__":
    #model_name = "ET"
    for model_name in ["LGB1", "XGB1"]:

        print("Model building..")
        kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
        cv_scores = []
        pred_test_full = 0
        pred_val_full = np.zeros(train_X.shape[0])
        for dev_index, val_index in kf.split(train_X, train_y):
            dev_X, val_X = train_X.iloc[dev_index,:], train_X.iloc[val_index,:]
            dev_y, val_y = train_y[dev_index], train_y[val_index]

            if model_name == "XGB1":
                pred_val, loss, pred_test = runXGB(dev_X, dev_y, val_X, val_y, test_X, rounds=5000, feature_names=dev_X.columns.tolist())
            elif model_name == "LGB1":
                pred_val, loss, pred_test = runLGB(dev_X, dev_y, val_X, val_y, test_X, rounds=5000)
            pred_val_full[val_index] = pred_val
            pred_test_full = pred_test_full + pred_test
            cv_scores.append(loss)
            print(cv_scores)
        pred_test_full /= 5.
        print(metrics.roc_auc_score(train_y, pred_val_full))

        out_df = pd.DataFrame({"transaction_id":test_id})
        out_df["target"] = pred_test_full
        out_df.to_csv("pred_test_v5_"+model_name+".csv", index=False)

        #out_df = pd.DataFrame({"transaction_id":train_id})
        #out_df["target"] = pred_val_full
        #out_df.to_csv("pred_val_v5_"+model_name+".csv", index=False)


Model building..
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.729034
[40]	valid_0's auc: 0.731389
[60]	valid_0's auc: 0.730362
[80]	valid_0's auc: 0.730175
[100]	valid_0's auc: 0.733212
[120]	valid_0's auc: 0.73054
[140]	valid_0's auc: 0.730753
[160]	valid_0's auc: 0.7314
[180]	valid_0's auc: 0.73138
[200]	valid_0's auc: 0.730942
Early stopping, best iteration is:
[100]	valid_0's auc: 0.733212
0.733211934982
[0.73321193498178339]
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.728689
[40]	valid_0's auc: 0.728316
[60]	valid_0's auc: 0.730217
[80]	valid_0's auc: 0.729941
[100]	valid_0's auc: 0.732037
[120]	valid_0's auc: 0.731101
[140]	valid_0's auc: 0.730531
[160]	valid_0's auc: 0.730056
[180]	valid_0's auc: 0.728113
[200]	valid_0's auc: 0.728602
Early stopping, best iteration is:
[102]	valid_0's auc: 0.732155
0.732154923006
[0.73321193498178339, 0.73215492300611085]
Training until validation scores don't impro

In [ ]:
#XGB 0.73633  0.731481622839              0.73463  0.731768245692
params["objective"] = "binary:logistic"
params['eval_metric'] = 'auc'
params["eta"] = 0.2
params["subsample"] = 0.7
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.7
params["max_depth"] = 10
params["silent"] = 1
#XGB 0.73594  0.730887790006
params["objective"] = "binary:logistic"
params['eval_metric'] = 'auc'
params["eta"] = 0.1
params["subsample"] = 0.7
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.7
params["max_depth"] = 8
params["silent"] = 1
params["seed"] = seed_val
#LGM 0.73414  0.731143283655
params["objective"] = "binary"
params['metric'] = 'auc'
params["max_depth"] = 8
params["min_data_in_leaf"] = 128
params["learning_rate"] = 0.1
params["bagging_fraction"] = 0.7
params["feature_fraction"] = 0.7
params["bagging_freq"] = 5
params["bagging_seed"] = seed_val
#LGM 0.73422  0.731471583671
params["objective"] = "binary"
params['metric'] = 'auc'
params["max_depth"] = 10
params["min_data_in_leaf"] = 128
params["learning_rate"] = 0.1
params["bagging_fraction"] = 0.7
params["feature_fraction"] = 0.7
params["bagging_freq"] = 5
params["bagging_seed"] = seed_val
#LGM   0.731282373295
params["objective"] = "binary"
params['metric'] = 'auc'
params["max_depth"] = 10
params["min_data_in_leaf"] = 128
params["learning_rate"] = 0.2
params["bagging_fraction"] = 0.7
params["feature_fraction"] = 0.7
params["bagging_freq"] = 5
params["bagging_seed"] = seed_val
#LGM   0.732104622404                        0.73373   0.730939456999
params["objective"] = "binary"
params['metric'] = 'auc'
params["max_depth"] = 10
params["min_data_in_leaf"] = 64
params["learning_rate"] = 0.1
params["bagging_fraction"] = 0.7
params["feature_fraction"] = 0.7
params["bagging_freq"] = 5
params["bagging_seed"] = seed_val
#LGM                                         0.73439      0.732129309669
params["objective"] = "binary"
params['metric'] = 'auc'
params["max_depth"] = 10
params["min_data_in_leaf"] = 64
params["learning_rate"] = 0.2
params["bagging_fraction"] = 0.7
params["feature_fraction"] = 0.7
params["bagging_freq"] = 5

In [31]:
from catboost import CatBoostClassifier
def runCatB(train_X, train_y, test_X, test_y=None, test_X2=None, depth=8):
    model = CatBoostClassifier(
                                iterations = 1000,
                                learning_rate = 0.2,
                                depth = depth,
                                od_type='Iter',
                                od_wait=100,
                                rsm=0.7,
                                l2_leaf_reg=3,
                                eval_metric = 'AUC', 
                                verbose=True,
                                random_seed=42)
    
    model.fit(train_X, train_y, eval_set=(test_X, test_y))#, plot=True)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth : ", depth)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

/home/dj/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.4 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [32]:
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
cv_scores = []
pred_test_full = 0
pred_val_full = np.zeros(train_X.shape[0])
for dev_index, val_index in kf.split(train_X, train_y):
    dev_X, val_X = train_X.iloc[dev_index,:], train_X.iloc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]

        
    pred_val, loss, pred_test = runCatB(dev_X, dev_y, val_X, val_y, test_X)
            
    pred_val_full[val_index] = pred_val
    pred_test_full = pred_test_full + pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.
print(metrics.roc_auc_score(train_y, pred_val_full))

/home/dj/anaconda3/lib/python3.6/site-packages/catboost/core.py:1240: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (posible values: 'Silent', 'Verbose', 'Info', 'Debug').
  super(CatBoostClassifier, self).__init__(params)


0: learn: 0.690253	test: 0.6872842	bestTest: 0.6872842 (0)	total: 2.6s	remaining: 43m 12s
1: learn: 0.7067968	test: 0.705925	bestTest: 0.705925 (1)	total: 3.1s	remaining: 25m 45s
2: learn: 0.7127781	test: 0.7098438	bestTest: 0.7098438 (2)	total: 3.63s	remaining: 20m 5s
3: learn: 0.7132002	test: 0.7113407	bestTest: 0.7113407 (3)	total: 4.27s	remaining: 17m 43s
4: learn: 0.7166818	test: 0.7145716	bestTest: 0.7145716 (4)	total: 4.86s	remaining: 16m 7s
5: learn: 0.7215131	test: 0.7192641	bestTest: 0.7192641 (5)	total: 5.42s	remaining: 14m 57s
6: learn: 0.7220094	test: 0.7192931	bestTest: 0.7192931 (6)	total: 6.1s	remaining: 14m 25s
7: learn: 0.7235972	test: 0.7200475	bestTest: 0.7200475 (7)	total: 6.78s	remaining: 14m
8: learn: 0.7268306	test: 0.7220191	bestTest: 0.7220191 (8)	total: 7.35s	remaining: 13m 29s
9: learn: 0.7278768	test: 0.724376	bestTest: 0.724376 (9)	total: 8.02s	remaining: 13m 14s
10: learn: 0.7282133	test: 0.724835	bestTest: 0.724835 (10)	total: 8.69s	remaining: 13m 1s
11:

89: learn: 0.7467302	test: 0.7323623	bestTest: 0.7323623 (89)	total: 53.8s	remaining: 9m 3s
90: learn: 0.746786	test: 0.7323457	bestTest: 0.7323623 (89)	total: 54.4s	remaining: 9m 3s
91: learn: 0.7468901	test: 0.7316069	bestTest: 0.7323623 (89)	total: 55s	remaining: 9m 2s
92: learn: 0.7470834	test: 0.7318884	bestTest: 0.7323623 (89)	total: 55.7s	remaining: 9m 2s
93: learn: 0.7471847	test: 0.7317612	bestTest: 0.7323623 (89)	total: 56.4s	remaining: 9m 3s
94: learn: 0.7474569	test: 0.7319369	bestTest: 0.7323623 (89)	total: 57s	remaining: 9m 3s
95: learn: 0.74752	test: 0.7318505	bestTest: 0.7323623 (89)	total: 57.8s	remaining: 9m 3s
96: learn: 0.7475676	test: 0.7319119	bestTest: 0.7323623 (89)	total: 58.5s	remaining: 9m 4s
97: learn: 0.7476581	test: 0.7320423	bestTest: 0.7323623 (89)	total: 59.1s	remaining: 9m 4s
98: learn: 0.7479456	test: 0.7323656	bestTest: 0.7323656 (98)	total: 59.7s	remaining: 9m 3s
99: learn: 0.7481616	test: 0.7326338	bestTest: 0.7326338 (99)	total: 1m	remaining: 9m 3

51: learn: 0.7414301	test: 0.7251147	bestTest: 0.7255299 (33)	total: 31s	remaining: 9m 25s
52: learn: 0.7416182	test: 0.7252331	bestTest: 0.7255299 (33)	total: 31.6s	remaining: 9m 24s
53: learn: 0.7422431	test: 0.72513	bestTest: 0.7255299 (33)	total: 32.2s	remaining: 9m 24s
54: learn: 0.7427678	test: 0.7256929	bestTest: 0.7256929 (54)	total: 32.9s	remaining: 9m 24s
55: learn: 0.7426857	test: 0.7256654	bestTest: 0.7256929 (54)	total: 33.5s	remaining: 9m 24s
56: learn: 0.7428418	test: 0.7256831	bestTest: 0.7256929 (54)	total: 34s	remaining: 9m 22s
57: learn: 0.7430866	test: 0.7257468	bestTest: 0.7257468 (57)	total: 34.6s	remaining: 9m 22s
58: learn: 0.7433008	test: 0.7257694	bestTest: 0.7257694 (58)	total: 35.3s	remaining: 9m 22s
59: learn: 0.7435062	test: 0.7253945	bestTest: 0.7257694 (58)	total: 35.8s	remaining: 9m 21s
60: learn: 0.7437025	test: 0.7254518	bestTest: 0.7257694 (58)	total: 36.5s	remaining: 9m 22s
61: learn: 0.7440862	test: 0.7257143	bestTest: 0.7257694 (58)	total: 37s	rem

139: learn: 0.7621605	test: 0.7265708	bestTest: 0.7267625 (138)	total: 1m 24s	remaining: 8m 41s
140: learn: 0.7627892	test: 0.7265833	bestTest: 0.7267625 (138)	total: 1m 25s	remaining: 8m 41s
141: learn: 0.7629334	test: 0.7267415	bestTest: 0.7267625 (138)	total: 1m 26s	remaining: 8m 39s
142: learn: 0.7630229	test: 0.7269216	bestTest: 0.7269216 (142)	total: 1m 26s	remaining: 8m 38s
143: learn: 0.7633951	test: 0.7270695	bestTest: 0.7270695 (143)	total: 1m 27s	remaining: 8m 38s
144: learn: 0.7634076	test: 0.7270697	bestTest: 0.7270697 (144)	total: 1m 27s	remaining: 8m 37s
145: learn: 0.763466	test: 0.7269557	bestTest: 0.7270697 (144)	total: 1m 28s	remaining: 8m 36s
146: learn: 0.7635444	test: 0.7268789	bestTest: 0.7270697 (144)	total: 1m 29s	remaining: 8m 36s
147: learn: 0.7636646	test: 0.726879	bestTest: 0.7270697 (144)	total: 1m 29s	remaining: 8m 36s
148: learn: 0.7642893	test: 0.7269747	bestTest: 0.7270697 (144)	total: 1m 30s	remaining: 8m 36s
149: learn: 0.7643894	test: 0.7269856	best

...

287: learn: 0.777647	test: 0.7311963	bestTest: 0.7315143 (275)	total: 2m 50s	remaining: 7m 1s
288: learn: 0.77766	test: 0.7312252	bestTest: 0.7315143 (275)	total: 2m 51s	remaining: 7m 1s
289: learn: 0.7777356	test: 0.7312965	bestTest: 0.7315143 (275)	total: 2m 51s	remaining: 7m
290: learn: 0.7777532	test: 0.7312642	bestTest: 0.7315143 (275)	total: 2m 52s	remaining: 6m 59s
291: learn: 0.7777566	test: 0.7312697	bestTest: 0.7315143 (275)	total: 2m 52s	remaining: 6m 59s
292: learn: 0.7777626	test: 0.7313002	bestTest: 0.7315143 (275)	total: 2m 53s	remaining: 6m 58s
293: learn: 0.7778206	test: 0.731349	bestTest: 0.7315143 (275)	total: 2m 54s	remaining: 6m 58s
294: learn: 0.7779061	test: 0.731371	bestTest: 0.7315143 (275)	total: 2m 54s	remaining: 6m 57s
295: learn: 0.7782206	test: 0.7312656	bestTest: 0.7315143 (275)	total: 2m 55s	remaining: 6m 57s
296: learn: 0.7783627	test: 0.7313017	bestTest: 0.7315143 (275)	total: 2m 55s	remaining: 6m 56s
297: learn: 0.7788094	test: 0.7311305	bestTest: 0.7

373: learn: 0.7833252	test: 0.7305231	bestTest: 0.7315143 (275)	total: 3m 39s	remaining: 6m 7s
374: learn: 0.7833725	test: 0.7305734	bestTest: 0.7315143 (275)	total: 3m 40s	remaining: 6m 6s
375: learn: 0.7836407	test: 0.7303301	bestTest: 0.7315143 (275)	total: 3m 40s	remaining: 6m 6s
376: learn: 0.783633	test: 0.7303341	bestTest: 0.7315143 (275)	total: 3m 41s	remaining: 6m 5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7315143435
bestIteration = 275

Depth :  8
Train and Test loss :  0.783632963102 0.730334079765
[0.73173024442866696, 0.72712842450385662, 0.7303340797652994]
0: learn: 0.6940769	test: 0.6889813	bestTest: 0.6889813 (0)	total: 467ms	remaining: 7m 46s
1: learn: 0.7047055	test: 0.6995002	bestTest: 0.6995002 (1)	total: 978ms	remaining: 8m 7s
2: learn: 0.7123612	test: 0.7076211	bestTest: 0.7076211 (2)	total: 1.48s	remaining: 8m 10s
3: learn: 0.7183592	test: 0.7152891	bestTest: 0.7152891 (3)	total: 2.03s	remaining: 8m 26s
4: learn: 0.7183256	test: 0.71

83: learn: 0.7471244	test: 0.7236056	bestTest: 0.725604 (42)	total: 48.8s	remaining: 8m 51s
84: learn: 0.7473206	test: 0.7230353	bestTest: 0.725604 (42)	total: 49.5s	remaining: 8m 53s
85: learn: 0.7474035	test: 0.723124	bestTest: 0.725604 (42)	total: 50.1s	remaining: 8m 52s
86: learn: 0.7475613	test: 0.7232881	bestTest: 0.725604 (42)	total: 50.7s	remaining: 8m 52s
87: learn: 0.74775	test: 0.7233921	bestTest: 0.725604 (42)	total: 51.3s	remaining: 8m 52s
88: learn: 0.7479041	test: 0.7236599	bestTest: 0.725604 (42)	total: 52s	remaining: 8m 52s
89: learn: 0.7479665	test: 0.7237616	bestTest: 0.725604 (42)	total: 52.6s	remaining: 8m 52s
90: learn: 0.7481939	test: 0.7238355	bestTest: 0.725604 (42)	total: 53.2s	remaining: 8m 51s
91: learn: 0.7490404	test: 0.724277	bestTest: 0.725604 (42)	total: 53.9s	remaining: 8m 52s
92: learn: 0.7492324	test: 0.7244968	bestTest: 0.725604 (42)	total: 54.5s	remaining: 8m 51s
93: learn: 0.7494144	test: 0.7246459	bestTest: 0.725604 (42)	total: 55s	remaining: 8m 

26: learn: 0.7333634	test: 0.7258173	bestTest: 0.7258173 (26)	total: 16.1s	remaining: 9m 38s
27: learn: 0.7338004	test: 0.7260871	bestTest: 0.7260871 (27)	total: 16.6s	remaining: 9m 36s
28: learn: 0.7339469	test: 0.7269529	bestTest: 0.7269529 (28)	total: 17.2s	remaining: 9m 35s
29: learn: 0.7347019	test: 0.7276017	bestTest: 0.7276017 (29)	total: 17.7s	remaining: 9m 32s
30: learn: 0.7347297	test: 0.7272911	bestTest: 0.7276017 (29)	total: 18.4s	remaining: 9m 33s
31: learn: 0.7355336	test: 0.72719	bestTest: 0.7276017 (29)	total: 18.9s	remaining: 9m 31s
32: learn: 0.7358399	test: 0.7270979	bestTest: 0.7276017 (29)	total: 19.5s	remaining: 9m 32s
33: learn: 0.7359887	test: 0.7270616	bestTest: 0.7276017 (29)	total: 20.2s	remaining: 9m 33s
34: learn: 0.7360015	test: 0.7270608	bestTest: 0.7276017 (29)	total: 20.7s	remaining: 9m 32s
35: learn: 0.7361354	test: 0.7270323	bestTest: 0.7276017 (29)	total: 21.4s	remaining: 9m 32s
36: learn: 0.7365462	test: 0.7276033	bestTest: 0.7276033 (36)	total: 22s

115: learn: 0.7560248	test: 0.72848	bestTest: 0.7294489 (92)	total: 1m 13s	remaining: 9m 17s
116: learn: 0.7563055	test: 0.7286533	bestTest: 0.7294489 (92)	total: 1m 13s	remaining: 9m 17s
117: learn: 0.756633	test: 0.7286242	bestTest: 0.7294489 (92)	total: 1m 14s	remaining: 9m 16s
118: learn: 0.756834	test: 0.728571	bestTest: 0.7294489 (92)	total: 1m 15s	remaining: 9m 15s
119: learn: 0.757009	test: 0.7285168	bestTest: 0.7294489 (92)	total: 1m 15s	remaining: 9m 15s
120: learn: 0.7574434	test: 0.7282824	bestTest: 0.7294489 (92)	total: 1m 16s	remaining: 9m 15s
121: learn: 0.7575935	test: 0.7283632	bestTest: 0.7294489 (92)	total: 1m 17s	remaining: 9m 14s
122: learn: 0.7577247	test: 0.7283805	bestTest: 0.7294489 (92)	total: 1m 17s	remaining: 9m 14s
123: learn: 0.7577952	test: 0.7283234	bestTest: 0.7294489 (92)	total: 1m 18s	remaining: 9m 13s
124: learn: 0.758034	test: 0.7281469	bestTest: 0.7294489 (92)	total: 1m 18s	remaining: 9m 12s
125: learn: 0.7587283	test: 0.7279918	bestTest: 0.7294489

In [ ]:
#0.73409
Depth :  8
learning_rate = 0.2
0.728183167395
#0.73516
Depth :  8
learning_rate = 0.2
mse=0.7
0.729139632577
#0.73356
Depth :  7
learning_rate = 0.2
mse=0.7
0.727873109999
#
Depth :  6
learning_rate = 0.2
mse=0.7
0.728397722185

In [33]:
out_df = pd.DataFrame({"transaction_id":test_id})
out_df["target"] = pred_test_full
out_df.to_csv("pred_test_v5_CatB_freq_Str(depth8-0.2-0.7).csv", index=False)

In [185]:
def runET(train_X, train_y, test_X, test_y=None, test_X2=None, depth=20, leaf=10, feat=0.8):
    model = ensemble.ExtraTreesClassifier(
                                        n_estimators = 100,
                                        criterion = 'gini',
                                        max_depth = depth,
                                        min_samples_split = 10,
                                        min_samples_leaf = leaf,
                                        warm_start = True,
                                        max_features =  feat,
                                        #min_impurity_split = 0.1,
                                        n_jobs = -1,
                                        verbose = 3,
                                        random_state = 0)
    model.fit(train_X, train_y)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth, leaf, feat : ", depth, leaf, feat)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

In [186]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2018)
cv_scores = []
pred_test_full = 0
pred_val_full = np.zeros(train_X.shape[0])
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.iloc[dev_index,:], train_X.iloc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]

        
    pred_val, loss, pred_test = runET(dev_X, dev_y, val_X, val_y, test_X)
            
    pred_val_full[val_index] = pred_val
    pred_test_full = pred_test_full + pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.
print(metrics.roc_auc_score(train_y, pred_val_full))

building tree 2 of 100
building tree 1 of 100building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   29.9s


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.7s finished


Depth, leaf, feat :  20 10 0.8
Train and Test loss :  0.858975958905 0.732088511145
[0.73208851114492313]
building tree 2 of 100building tree 1 of 100building tree 3 of 100


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


...

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   30.7s


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.2s


Depth, leaf, feat :  20 10 0.8
Train and Test loss :  0.853413704547 0.72798137251
[0.73208851114492313, 0.72993739601453644, 0.72331505105579152, 0.72541433735838246, 0.72798137251044004]
0.727715212136


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.9s finished


In [187]:
out_df = pd.DataFrame({"transaction_id":test_id})
out_df["target"] = pred_test_full
out_df.to_csv("pred_test_v5_ET.csv", index=False)

In [199]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
def runADA(train_X, train_y, test_X, test_y=None, test_X2=None, depth=6):
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth, max_features=0.7, min_samples_leaf=5),
                               n_estimators = 300,
                               learning_rate = 0.2,
                               random_state=42)
    
    model.fit(train_X, train_y)#, eval_set=(test_X, test_y))#, plot=True)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth : ", depth)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

In [200]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2018)
cv_scores = []
pred_test_full = 0
pred_val_full = np.zeros(train_X.shape[0])
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.iloc[dev_index,:], train_X.iloc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]

        
    pred_val, loss, pred_test = runADA(dev_X, dev_y, val_X, val_y, test_X)
            
    pred_val_full[val_index] = pred_val
    pred_test_full = pred_test_full + pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.
print(metrics.roc_auc_score(train_y, pred_val_full))

Depth :  6
Train and Test loss :  0.832458583785 0.710091525144
[0.7100915251436426]
Depth :  6
Train and Test loss :  0.834312921178 0.709692699578
[0.7100915251436426, 0.70969269957816228]
Depth :  6
Train and Test loss :  0.833311675136 0.709264199389
[0.7100915251436426, 0.70969269957816228, 0.70926419938928842]
Depth :  6
Train and Test loss :  0.83326242463 0.70249562109
[0.7100915251436426, 0.70969269957816228, 0.70926419938928842, 0.70249562108972763]
Depth :  6
Train and Test loss :  0.835442925819 0.707837409163
[0.7100915251436426, 0.70969269957816228, 0.70926419938928842, 0.70249562108972763, 0.70783740916310123]
0.7078584866
